In [ ]:
#getting data files path
train_path = "/content/hin_train.csv"
val_path = "/content/hin_valid.csv"
test_path = "/content/hin_test.csv"

In [ ]:
# importing libraries
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import torch
import random
import torchvision
import zipfile
from torchvision.datasets.utils import download_url
from torch.utils.data import random_split
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor
from torchvision import transforms 
import torch.nn as nn
import torch.optim as optim
import csv
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.1/205.1 kB 19.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.5 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8791 sha256=686a9e89b57ebe9095639f07f4c6e21d31bbfd487308d7a589c6237ddae3e84e
  Stored in directory: /root/.cache/pip/wheels/e7/f3/22/152153d6eb222ee7a56ff8617d80ee5207207a8c00a7aab794
Successfully built pathtools


In [ ]:
import wandb

In [ ]:
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
!pip install GPUtil
import torch
from numba import cuda
from GPUtil import showUtilization as gpu_usage

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for GPUtil: filename=GPUtil-1.4.0-py3-none-any.whl size=7393 sha256=19e75e39fc63d8c69578753f0c33df092712dbe2c0b998e6ef5438a14bb5ab61
  Stored in directory: /root/.cache/pip/wheels/a9/8a/bd/81082387151853ab8b6b3ef33426e98f5cbfebc3c397a9d4d0
Successfully built GPUtil


## **Data PreProcessing**

In [ ]:
def getWords(path):
        
        hindi = []
        english = []

        file = open(path)
        dataset = csv.reader(file, delimiter = ",")

        # to get the words in a list

        for data in dataset:
          e=data[0]
          h=data[1]
          english.append(e)
          hindi.append(h)

        #appending the start and end characters to hindi words
        for i in range(len(hindi)):
            hindi[i] = "\t" + hindi[i] +"\n"

        return np.array(hindi), np.array(english)

In [ ]:
def getChar(data):
      data_char = set() #to store the the unique characters present in data
      data_char.add(" ")
      for word in data:
        for char in word:
          if char not in data_char:
            data_char.add(char)

      #sort the characters in dataset
      data_char = sorted(list(data_char))

      #number of characters in the set
      num_tokens = len(data_char)

      #get the max length of the words
      max_len = max([len(word) for word in data])

      #return set of all characters in data
      return data_char, num_tokens, max_len 

In [ ]:
def getData(english, hindi, max_enc_len, max_dec_len, num_decoder_tokens, input_token_index, target_token_index):
      #initializing with 0s for max_length
      encoder_input_data = np.zeros((len(english), max_enc_len), dtype="float32") 
      decoder_input_data = np.zeros((len(english), max_dec_len), dtype="float32") 

      #creating indices for characters that exist
      for i, (english, hindi) in enumerate(zip(english, hindi)):
          for t, char in enumerate(english):
              encoder_input_data[i, t] = input_token_index[char]
          
          for t, char in enumerate(hindi):
              decoder_input_data[i, t] = target_token_index[char]       
          decoder_input_data[i, t+1:] = target_token_index[' ']

      return encoder_input_data, decoder_input_data

In [ ]:
def createDictionary(input_tokens, target_tokens):
    #making a dictionary to map the characters with the indices
    input_token_index = dict([(char, i) for i, char in enumerate(input_tokens)])
    target_token_index = dict([(char, i) for i, char in enumerate(target_tokens)])
    
    #making a dictionary to map the indices with the characters
    reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
    reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())
    return input_token_index,target_token_index,reverse_input_char_index,reverse_target_char_index

In [ ]:
#get the training words as an array from train data
train_hindi_words, train_english_words =getWords(train_path)
#get validation words
val_hindi_words, val_english_words =getWords(val_path)
#get test words
test_hindi_words, test_english_words =getWords(test_path)

  
#get the characters from train and val and test dataset
train_eng_characters, train_num_encoder_tokens, train_max_enc_len = getChar(train_english_words)
train_hin_characters, train_num_decoder_tokens, train_max_dec_len = getChar(train_hindi_words)

val_eng_characters, val_num_encoder_tokens, val_max_enc_len = getChar(val_english_words)
val_hin_characters, val_num_decoder_tokens, val_max_dec_len = getChar(val_hindi_words)

test_eng_characters, test_num_encoder_tokens, test_max_enc_len = getChar(test_english_words)
test_hin_characters, test_num_decoder_tokens, test_max_dec_len = getChar(test_hindi_words)

#take the largest number of tokens and max_length of words on both encoder and decoder
num_encoder_tokens = max(val_num_encoder_tokens, train_num_encoder_tokens,test_num_encoder_tokens)
num_decoder_tokens = max(val_num_decoder_tokens, train_num_decoder_tokens,test_num_decoder_tokens)

max_enc_len = max(train_max_enc_len, val_max_enc_len,test_max_enc_len)
max_dec_len = max(train_max_dec_len, val_max_dec_len,test_max_dec_len)

#finding the set having unique characters from train, val,test data
input_hin_characters=set()
for char in train_hin_characters:
  input_hin_characters.add(char)
        
for char in val_hin_characters:
  input_hin_characters.add(char)
        
for char in test_hin_characters:
  input_hin_characters.add(char)
        

# calling createDictionary function to make a dictionary and reverse dictionary to map the characters with the indices and indices to characters
input_token_index,target_token_index,reverse_input_char_index,reverse_target_char_index=createDictionary(train_eng_characters,list(input_hin_characters)) 

#creating input data for encoder and decoder to process in batch
train_encoder_input_data, train_decoder_input_data = getData(train_english_words, train_hindi_words, max_enc_len, max_dec_len, num_decoder_tokens, input_token_index, target_token_index)
val_encoder_input_data, val_decoder_input_data = getData(val_english_words, val_hindi_words, max_enc_len, max_dec_len, num_decoder_tokens, input_token_index, target_token_index)
test_encoder_input_data, test_decoder_input_data = getData(test_english_words, test_hindi_words, max_enc_len, max_dec_len, num_decoder_tokens, input_token_index, target_token_index)

# converting to tensors from numpy
train_encoder_input_data=torch.from_numpy(train_encoder_input_data).to(device).long()
train_decoder_input_data=torch.from_numpy(train_decoder_input_data).to(device).long()

val_encoder_input_data=torch.from_numpy(val_encoder_input_data).to(device).long()
val_decoder_input_data=torch.from_numpy(val_decoder_input_data).to(device).long()

test_encoder_input_data=torch.from_numpy(test_encoder_input_data).to(device).long()
test_decoder_input_data=torch.from_numpy(test_decoder_input_data).to(device).long()


##**Encoder Part**

In [ ]:
class Encoder(nn.Module):
    def __init__(self,cell,embedding_size,hidden_size,no_of_layers,dropout):
        super(Encoder,self).__init__()
        #setting the encoder part
        self.hidden_size=hidden_size
        self.input_token_index_len=len(input_token_index)
        self.no_of_layers = no_of_layers
        self.embedding_size = embedding_size
        self.cell=cell
        self.train_eng_characters=train_eng_characters
        self.drop = nn.Dropout(dropout) # using dropout
        #creating an embedding layer
        self.encoder_embedding = nn.Embedding(self.input_token_index_len,self.embedding_size).to(device)
        self.gru = nn.GRU(self.embedding_size,self.hidden_size,self.no_of_layers,batch_first = True,bidirectional = True).to(device)
        self.rnn = nn.RNN(self.embedding_size,self.hidden_size,self.no_of_layers,batch_first = True,bidirectional = True).to(device)
        self.lstm = nn.LSTM(self.embedding_size,self.hidden_size,self.no_of_layers,batch_first = True,bidirectional = True).to(device)
    
    #forward function for encoder
    def forward(self,input,hidden,cell):
        enc_embedd= self.encoder_embedding(input)
        temp=enc_embedd
        #using dropout
        enc_embedd = self.drop(temp)
        # RNN/GRU/LSTM layer of the encoder
        if(self.cell == 'RNN'):
            input1=enc_embedd
            input2=hidden
            output,hidden = self.rnn(input1,input2)
        elif(self.cell == 'GRU'):
            input1=enc_embedd
            input2=hidden
            output,hidden = self.gru(input1,input2)
        elif(self.cell == 'LSTM'):
            input1=enc_embedd
            output,(hidden,cell) = self.lstm(input1,(hidden,cell))
        return output,(hidden,cell)

## **Decoder Part**

In [ ]:
class Decoder(nn.Module):
    def __init__(self,cell,embedding_size,hidden_size,no_of_layers,dropout,batchsize):
        super(Decoder,self).__init__()
        #setting the encoder part
        self.cell=cell
        self.no_of_layers = no_of_layers
        self.target_token_index_len=len(target_token_index)
        self.drop = nn.Dropout(dropout)  #using dropout
        self.embedding_size = embedding_size
        self.train_hin_characters=train_hin_characters
        self.hidden_size=hidden_size
        self.batchsize=batchsize
        #creating an embedding layer
        self.decoder_embedding = nn.Embedding(self.target_token_index_len,self.embedding_size).to(device)
        input1=embedding_size+ hidden_size*2
        self.lstm = nn.LSTM(input1,hidden_size,self.no_of_layers,batch_first = True).to(device)
        input2=embedding_size + hidden_size*2
        self.rnn = nn.RNN(input2,hidden_size,self.no_of_layers,batch_first = True).to(device)
        input3=embedding_size+ hidden_size*2
        self.gru = nn.GRU(input3,hidden_size,self.no_of_layers,batch_first = True).to(device)
        input4=self.target_token_index_len
        self.linear = nn.Linear(self.hidden_size,input4,bias=True).to(device)
        self.softmax = nn.Softmax(dim = 2).to(device)

    #using forward function for decoder
    def forward(self,input,hidden,cell,hiddenj):
        dec_embedd = self.decoder_embedding(input)
        temp1=hiddenj[0].reshape(self.batchsize,1,self.hidden_size)
        temp2=hiddenj[1].reshape(self.batchsize,1,self.hidden_size)
        dec_embedd=torch.cat((dec_embedd,temp1,temp2),dim=2)
        dec_embedd=self.drop(dec_embedd)
        # RNN/GRU/LSTM layer of the decoder
        if(self.cell == 'LSTM'):
            input1=dec_embedd
            output,(hidden,cell) = self.lstm(dec_embedd,(hidden,cell))
        elif(self.cell == 'RNN'):
            input1=dec_embedd
            input2=hidden
            output,hidden = self.rnn(input1,input2)
        elif(self.cell == 'GRU'):
            input1=dec_embedd
            input2=hidden
            output,hidden = self.gru(input1,input2)
        output = self.linear(output)
        temp=(hidden,cell)
        return output,temp

## **Helper Functions**

In [ ]:
#function to get batchsize of data
def getBatchData(input,batchsize,x):
  return input[x:x+batchsize]

# function to do split as output is also biredictional output as encoder is bidirectional. So we are splitting it.
def getSplit(output,hidden_size):
  temp=torch.split(output,[hidden_size,hidden_size],dim = 2)
  input1=temp[0]
  input2=temp[1]
  temp=torch.add(input1,input2)/2
  return temp
  
# function to do resize as encoder is bidirectional asnd decoder is unidirectional. so we are reshaping it.
def getResize(var,val,no_of_layers,batchsize,hidden_size):
  var=var.reshape(2,no_of_layers,batchsize,hidden_size)
  var=torch.add(var[0],var[1])/2
  return var

#function to concatenate and return as tuple
def getConcatenation(lst):
  temp= torch.cat(tuple(x for x in lst),dim =1).to(device)
  return temp

In [ ]:
#function to calculate accuracy
def getAccuracy(target,output):
    n=len(target)
    total = 0
    i=0
    while i<n:
        if(torch.equal(target[i],output[i])):
            total += 1
        i+=1
    return total

## **Function for validation data**

In [ ]:
def getValidation(attention,val_encoder_input_data,val_decoder_input_data,encoder,decoder,batchsize,hidden_size,embedding_size,no_of_layers):
    with torch.no_grad():
        temp=torch.zeros(2*no_of_layers,batchsize,hidden_size).to(device)
        enc_hidden = temp
        enc_cell = temp
        total_loss=0
        total_acc=0
        for x in range(0,len(val_encoder_input_data),batchsize):
            y_pred=[]
            y_actual=[]
            loss=0
            #input for encoder
            input_tensor = getBatchData(val_encoder_input_data,batchsize,x).to(device)
            #calling encoder forward function
            output,(hidden,cell) = encoder.forward(input_tensor,enc_hidden,enc_cell)
            output = getSplit(output,hidden_size)
            #input for first hidden state in decoder
            input =val_decoder_input_data[x:x+batchsize,0].to(device).reshape(batchsize,1)
            #calling resize because encoder is bidirectional
            hidden = getResize(hidden,2,no_of_layers,batchsize,hidden_size)
            cell = getResize(cell,2,no_of_layers,batchsize,hidden_size)
            hiddenj = hidden
            k=0
            temp1=output if attention else hiddenj
            #ruuning while loop for max no of characters in decoder data
            while k<max_dec_len:
                output1,(hidden,cell) = decoder.forward(input,hidden,cell,temp1)
                output2 = decoder.softmax(output1)
                output3 = torch.argmax(output2,dim = 2)
                y_pred.append(output1) # storing decoder predicted value
                y_actual.append(output3)# storing decoder predicted value index
                input = output3
                k+=1
            #flattening the tensor
            y_pred = getConcatenation(y_pred).reshape(max_dec_len*batchsize,len(target_token_index))
            y_actual = getConcatenation(y_actual)
            var=val_decoder_input_data[x:x+batchsize]
            total_acc += getAccuracy(var.to(device),y_actual) #calling accuracy function to get accuracy
            #calculating loss
            loss  = nn.CrossEntropyLoss(reduction = 'sum')(y_pred,var.reshape(-1).to(device))
            with torch.no_grad():
                total_loss += loss.item()
        denom=(len(val_decoder_input_data)*max_dec_len)
        val_loss = total_loss/denom  # calculating val loss
        total_loss=0
        #calculating val accuracy
        val_accuracy = (total_acc/len(val_decoder_input_data))
        val_accuracy=val_accuracy*100
        total_acc=0
        return val_loss,val_accuracy

## **Train Function**

In [ ]:
def train(rnn,batchsize,hidden_size,embedding_size,no_of_layers,dropout,epochs):
    teacher_ratio = 0.5
    total_loss=0
    total_acc=0
    learning_rate=0.001
    #creating encoder and decoder model
    encoder = Encoder(rnn,embedding_size,hidden_size,no_of_layers,dropout).to(device)
    decoder = Decoder(rnn,embedding_size,hidden_size,no_of_layers,dropout,batchsize).to(device)
    opt_encoder = optim.Adam(encoder.parameters(),learning_rate)
    opt_decoder  = optim.Adam(decoder.parameters(),learning_rate)
    loss=0
    #running for loop epochs no of times
    for i in range(epochs):
        print('Epoch---',i+1,end=" ")
        #getting batchsize amount of encoder data
        for x in range(0,len(train_encoder_input_data),batchsize):
            y_pred = [] # to store predicted value by decoder to calculate loss
            y_val = []  #to store index of predicted value by decoder to calculate accuracy
            hidden_input = torch.zeros(no_of_layers,batchsize,hidden_size).to(device) # initial input to first hidden state(S0)
            temp=torch.zeros(2*no_of_layers,batchsize,hidden_size).to(device)
            input_tensor = getBatchData(train_encoder_input_data,batchsize,x).to(device) # to get batchsize of train data for encoder
            input_tensor_size=input_tensor.size()[0]
            #initializing with zeroes
            enc_hidden =temp
            enc_cell = temp
            if(batchsize>input_tensor_size):
              break
            #calling forward function of encoder
            output,(hidden,cell) = encoder.forward(input_tensor,enc_hidden,enc_cell)
            input = train_decoder_input_data[x:x+batchsize,0].to(device).reshape(batchsize,1) # to get batchsize amount of decoder input data for first hidden state
            #doing the resizeing for bidirectional implementation
            hidden = doResize(hidden,2,no_of_layers,batchsize,hidden_size)
            cell = doResize(cell,2,no_of_layers,batchsize,hidden_size)
            hiddenj = hidden
            #using teacher forcing 
            teacher_forcing = True if random.random() < teacher_ratio else False
            k=0
            if teacher_forcing:
                #runing while loop for max no of characters in decoder data
                while k<max_dec_len:
                    output1,(hidden,cell) = decoder.forward(input,hidden,cell,hiddenj) # calling decoder forward
                    output2 = decoder.softmax(output1)
                    output3 = torch.argmax(output2,dim = 2)
                    y_pred.append(output1) # storing decoder predicted value
                    y_val.append(output3) # storing decoder predicted value index
                    input = train_decoder_input_data[x:x+batchsize,i].to(device).reshape(batchsize,1) # giving groung truth data for next input
                    k+=1
            else:
                while k<max_dec_len:
                    output1,(hidden,cell) = decoder.forward(input,hidden,cell,hiddenj)  # calling decoder forward
                    output2 = decoder.softmax(output1)
                    output3 = torch.argmax(output2,dim = 2)
                    y_pred.append(output1)  # storing decoder predicted value
                    y_val.append(output3)   # storing decoder predicted value index
                    input = output3  # giving last output as next input
                    k+=1
            #flattening the tensor     
            y_pred = getConcatenation(y_pred).to(device).reshape(max_dec_len*batchsize,len(target_token_index)) # 
            y_val = getConcatenation(y_val)
            var=getBatchData(train_decoder_input_data,batchsize,x)
            total_acc += getAccuracy(var.to(device),y_val) #calling accuracy function to get accuracy
            #calculating loss
            loss = nn.CrossEntropyLoss(reduction = 'sum')(y_pred,var.reshape(-1).to(device))
            with torch.no_grad():
              total_loss += loss.item()
            loss.backward(retain_graph = True)
            #clipping the gradient
            torch.nn.utils.clip_grad_norm_(encoder.parameters(),max_norm = 1)
            torch.nn.utils.clip_grad_norm_(decoder.parameters(),max_norm = 1)
            opt_encoder.step()
            opt_decoder.step()
            opt_encoder.zero_grad()
            opt_decoder.zero_grad()
            loss=0
        del(y_pred)
        del(y_val)
        del(input)
        del(output1)
        del(output2)
        del(output3)
        del(hidden)
        del(cell)
        del(hiddenj)
        del(output)
        denom=(len(train_decoder_input_data)*max_dec_len)
        training_loss = total_loss/denom # calculating training loss
        #calculating training accuracy
        training_accuracy = (total_acc/len(train_decoder_input_data))*100
        #calling getValidation to get validation loss and accuracy
        validation_loss,validation_accuracy = getValidation(False,val_encoder_input_data,val_decoder_input_data,encoder,decoder,batchsize,hidden_size,embedding_size,no_of_layers)
        print(' Train loss = ',training_loss,' Train accuracy = ',training_accuracy,'   validation loss= ',validation_loss,'  validation accuaracy= ',validation_accuracy)
        total_loss=0
        total_acc=0

In [ ]:
#function for sweep without attention
def vanilla():
    wandb.init(project='DL-Assignment3')
    config = wandb.config
    wandb.run.name = "withoutattention_cell_type_{}_bidirec_{}_layers_{}_batchsize_{}_hidden_{}".format(config.cell_type,config.bidirectional,config.no_of_layers,config.batchsize,config.hidden_size)
    epochs = config.epochs
    gpu_usage()  
    hidden_size = config.hidden_size
    embedding_size = config.input_embedding_size
    dropout = config.dropout
    gpu_usage()
    no_of_layers = config.no_of_layers
    batchsize = config.batchsize
    rnn = config.cell_type
    #calling train function
    Encoder1,Decoder1 = train(rnn,batchsize,hidden_size,embedding_size,no_of_layers,dropout,epochs)
    gpu_usage()                             
    torch.cuda.empty_cache()
    cuda.select_device(0)
    cuda.close()
    cuda.select_device(0)
    gpu_usage()

In [ ]:
sweep_configuration = {
    'method' : 'bayes',
    'metric' : { 'goal' : 'maximize',
    'name' : 'validation_accuracy'},
    'parameters':{
        'cell_type' : {'values' : ['LSTM','RNN','GRU']},
        'batchsize' : {'values' : [128,256]},
        'input_embedding_size' : {'values' : [128,256,512,1024]},
        'dropout' : {'values' : [0.1,0.2,0.3,0.4,0.5]},
        'no_of_layers' : {'values' : [2,3,4]},
        'hidden_size' : {'values' : [128,256,512,1024]},
        'bidirectional' : {'values' : ['Yes']},
        'epochs' : {'values' : [10,20,30]}
    }
}
sweep_id = wandb.sweep(sweep = sweep_configuration,project = 'DL-Assignment3')
wandb.agent(sweep_id,function=vanilla,count = 10)